In [1]:
# import packages
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import os
import tensorflow as tf
import numpy as np
# envファイル読み込み
load_dotenv()
login_server = os.environ.get("LOGIN_SERVER")
login_id = int(os.environ.get("LOGIN_ID"))
login_password = os.environ.get("LOGIN_PASSWORD")

# login mt5
if not mt5.initialize():
    print("initialize() failed, error code =", mt5.last_error())
    quit()
print(f'MetaTrader5 version : {mt5.version()}')

authorized=mt5.login(login = login_id, password = login_password, server = login_server)
if authorized:
    account_info=mt5.account_info()
    if account_info!=None:
        print(account_info)
        account_info_dict = mt5.account_info()._asdict()
        for prop in account_info_dict:
            print("  {}={}".format(prop, account_info_dict[prop]))
        print()
else:
    print("failed to connect to trade account 25115284 with password=gqz0343lbdm, error code =",mt5.last_error())
    quit()
 
# 各種設定
symbol = "USDJPY"   # 取引対象
magic_number = 10001 # bot識別番号
per     = 0.1       # 1回の投資金額
losscut = 0.995     # ロスカットする比率
per_lot = 100000     # 1ロット当たりの通貨数
leverage = 100        # レバレッジ(整数)
maxlen = 60         # modelの入力データの長さ

# 最小lot数取得
point=mt5.symbol_info(symbol).point


MetaTrader5 version : (500, 3746, '21 May 2023')
AccountInfo(login=5013584284, trade_mode=0, leverage=500, limit_orders=200, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=0, fifo_close=False, balance=98738.0, credit=0.0, profit=0.0, equity=98738.0, margin=0.0, margin_free=98738.0, margin_level=0.0, margin_so_call=50.0, margin_so_so=30.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='python fx-trading', server='MetaQuotes-Demo', currency='JPY', company='MetaQuotes Software Corp.')
  login=5013584284
  trade_mode=0
  leverage=500
  limit_orders=200
  margin_so_mode=0
  trade_allowed=True
  trade_expert=True
  margin_mode=2
  currency_digits=0
  fifo_close=False
  balance=98738.0
  credit=0.0
  profit=0.0
  equity=98738.0
  margin=0.0
  margin_free=98738.0
  margin_level=0.0
  margin_so_call=50.0
  margin_so_so=30.0
  margin_initial=0.0
  margin_maintenance=0.0
  assets=0.0
  liabilities=0.0

In [12]:

balance = mt5.account_info()._asdict()["balance"]
rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_H1, 1, 60+maxlen-1)
rates_frame = pd.DataFrame(rates)
symbol_tick=mt5.symbol_info_tick(symbol)
result = mt5.order_send({
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": round((balance * per * leverage / rates_frame["close"].values[-1]) / per_lot, 2),
        "type": mt5.ORDER_TYPE_BUY,
        "price": symbol_tick.ask,
        "sl": rates_frame["close"].values[-1] * losscut,
        "tp": rates_frame["close"].values[-1] * 1.01,
        "deviation": 20,
        "magic": 234000,
        "comment": "python script open",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
        })
print(result)

OrderSendResult(retcode=10009, deal=50462060318, order=50462227175, volume=0.07, price=138.461, bid=138.461, ask=138.461, comment='Request executed', request_id=840728513, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='USDJPY', volume=0.07, price=138.461, stoplimit=0.0, sl=137.81744999999998, tp=139.89509999999999, deviation=20, type=0, type_filling=2, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


In [14]:
positions = mt5.positions_get(symbol = symbol)
position = positions[0]._asdict()
symbol_tick=mt5.symbol_info_tick(symbol)
print(position)
result = mt5.order_send({
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": position["symbol"],
        "volume": position["volume"],
        "type": mt5.ORDER_TYPE_SELL,
        "position": position["ticket"],
        "price": symbol_tick.bid,
    })
print(result)

{'ticket': 50462227175, 'time': 1684851455, 'time_msc': 1684851455851, 'time_update': 1684851455, 'time_update_msc': 1684851455851, 'type': 0, 'magic': 234000, 'identifier': 50462227175, 'reason': 3, 'volume': 0.07, 'price_open': 138.461, 'sl': 137.817, 'tp': 139.895, 'price_current': 138.457, 'swap': 0.0, 'profit': -28.0, 'symbol': 'USDJPY', 'comment': 'python script op', 'external_id': ''}
OrderSendResult(retcode=10009, deal=50462060771, order=50462227654, volume=0.07, price=138.457, bid=138.457, ask=138.462, comment='Request executed', request_id=840728515, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='USDJPY', volume=0.07, price=138.457, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=1, type_filling=0, type_time=0, expiration=0, comment='', position=50462227175, position_by=0))
